In [2]:
import numpy as np 

In [3]:
def calcSVD(M):
    #-----U-------------
    t = M.transpose() 
    # transpose M
    MMT = np.dot(M,t)
    #multiply M * M^T 
    # # I picked dot because speed is more important in building a reccomendation system. 
    eigenvalues_MMT, eigenvectors_MMT = np.linalg.eig(MMT)
    # find the eigenvalues and vectors
    b_MMT = np.argsort(eigenvalues_MMT)[::-1]
    # argsort returns the indices of the sorted array 
    # sort it in descending order. 
    U = eigenvectors_MMT[:,b_MMT]
    # then, switch the first and second columns 

    #-------V-------------
    MTM = np.dot(t,M)
    eigenvalues_MTM, eigenvectors_MTM = np.linalg.eig(MTM)
    b_MTM= np.argsort(eigenvalues_MTM)[::-1]
    V_T = eigenvectors_MTM[:,b_MTM].transpose() 

    #--------S------------
    # We want to use the smaller matrix, MMT or MTM
    if np.size(MMT)> np.size(MTM): 
        eigenvalues = np.sqrt(eigenvalues_MTM)
        S = eigenvalues[::-1]
    else: 
        eigenvalues = np.sqrt(eigenvalues_MMT)
        S = eigenvalues[::-1]
    
    return S, V_T, U 

In [4]:
import pandas as pd

In [5]:
A = pd.read_csv('PCA_data.csv')
# This is the cleaned, and reduced data. Dimensionality reduction is very important because we want the computation of the reccomender system to be faster. 

In [6]:
A = np.array(A)

In [7]:
A.shape

(113999, 10)

In [8]:
A = A[:3999]
# reduced the rows that we use for computation reasons. 

In [9]:
#Calling the corresponding Fuctions and saving the values in variables 
Sigma, Vt,U  = calcSVD(A)

In [10]:
print("-------------------U-------------------")
print(U)
print("\n--------------Sigma----------------")
print(Sigma)
print("\n-------------V transpose---------------")
print(Vt)

-------------------U-------------------
[[ 3.36764531e-02+0.j         -8.54643989e-03+0.j
   2.16050679e-02+0.j         ... -8.10162558e-03-0.00239078j
  -8.10162558e-03+0.00239078j -2.85495115e-05+0.j        ]
 [ 2.94368637e-02+0.j         -4.22278738e-03+0.j
   1.46561339e-02+0.j         ... -1.41153240e-02-0.00437852j
  -1.41153240e-02+0.00437852j  5.43240688e-03+0.j        ]
 [ 4.88951963e-02+0.j          1.60272884e-02+0.j
  -6.49300557e-03+0.j         ... -3.15920354e-02-0.00196867j
  -3.15920354e-02+0.00196867j  4.86560615e-02+0.j        ]
 ...
 [-1.06463628e-03+0.j          7.80483455e-03+0.j
  -1.21839366e-02+0.j         ...  1.54115713e-02+0.01183537j
   1.54115713e-02-0.01183537j -6.97570926e-03+0.j        ]
 [ 9.49483571e-03+0.j         -1.09350905e-02+0.j
   5.92620882e-03+0.j         ... -9.02248793e-03-0.00919082j
  -9.02248793e-03+0.00919082j  5.08381617e-03+0.j        ]
 [-3.01078970e-03+0.j         -4.16914745e-03+0.j
  -3.16149523e-02+0.j         ... -1.42260246e-03+

In [30]:
def cosine_similarity(v,u):
    return (v @ u)/ (np.linalg.norm(v) * np.linalg.norm(u))
# multiply v and u together and divide by the norm of v and u 

In [67]:
U.shape

(3999, 3999)

In [60]:

# we want to find the rows of the data that are most similar to a given song. Thus, we will use the matrix U to find songs that are most similar. 
# Id is the index of the row that we are trying to find songs most similar to. 
# will generate top 3 highest song choices. 
def find_reccs(id=0):
    highest_similarity = float('-inf')
    second_similarity = float('-inf')
    third_similarity = float('-inf')
    highest_sim_col = -1
    second_row = -1 
    third_row = -1  
    for r in range(0,U.shape[1]):
        if r!= id: 
            similarity = cosine_similarity(U[:,id], U[:,r])
            # find the cosine similarity between the row I am working on and the row I am iteratinf through
            if similarity > highest_similarity:
                highest_similarity = similarity
                highest_sim_col = r
            if similarity > second_similarity and similarity != highest_similarity: 
                second_similarity = similarity
                second_row = r 
            if similarity > third_similarity and similarity != highest_similarity and similarity != second_similarity: 
                third_similarity = similarity
                third_row = r 
    return highest_sim_col,second_row,third_row

In [61]:
a,b,c = find_reccs(3)

In [62]:
a,b,c

(7, 3819, 103)

In [63]:
data = pd.read_csv('dataset.csv')

In [64]:
print('The top 3 songs that I recommend are most similar to:', data.loc[3]['track_name'])
print('1: ',data.loc[a]['track_name'],'by: ',data.loc[a]['artists'])
print('2: ',data.loc[b]['track_name'],'by: ',data.loc[b]['artists'])
print('3: ',data.loc[c]['track_name'],'by: ',data.loc[c]['artists'])


The top 3 songs that I reccommend are most similar to: Can't Help Falling In Love
1:  I'm Yours by:  Jason Mraz
2:  I Wish I Missed My Ex by:  Mahalia
3:  Mirrors by:  Boyce Avenue;Fifth Harmony
